In [2]:
pip install --upgrade pip

Note: you may need to restart the kernel to use updated packages.


# Tutoriel : rotation d'un qubit - rotation of a qubit

Dans l'exercice suivant, nous souhaitons implémenter un circuit effectuant une roation d'un qubit autours de l'axe X puis autours de l'axe Y.
L'idée sera ensuite de mesurer le gradient lié à ce circuit, puis de trouver une rotation optimale permettant de faire passer le qubit de l'état $|0\rangle$ à l'état $|1\rangle$.

\- \- \-

In the following exercise, we want to implement a circuit that rotates a qubit around the X axis and then around the Y axis.
The idea will then be to measure the gradient linked to this circuit, then to find an optimal rotation allowing the qubit to pass from the state $|0\rangle$ to the state $|1\rangle$.

### 1) Définition du circuit - Circuit Definition



In [3]:
import pennylane as qml

In [4]:
from pennylane import numpy as np

In [5]:
# Utilisation des modèles de qubit avec un seul qubit (wires=1) 
dev1 = qml.device("default.qubit", wires=1)

Définissons notre circuit :

\- \- \- 

We can define the circuit :

In [6]:
@qml.qnode(dev1)
def circuit(params):
    qml.RX(params[0], wires = 0) 
    qml.RY(params[1], wires = 0)
    return qml.expval(qml.PauliZ(0))

En choisissant comme angle de rotation 0.54 pour la première rotation et 0.12 pour la seconde, nous obtenons le résultat suivant :

\- \- \-

By choosing as rotation angle 0.54 for the first rotation and 0.12 for the second, we obtain the following result:

In [7]:

print(circuit([0.54, 0.12]))

0.8515405859048367


### 2) Calcul du gradient - Gradient calculation

Grâce à Pennylane, nous pouvons accéder au gradient de la fonction _circuit_ définie ci-dessus.

Nous pouvons différencier en utilisant la grad() fonction intégrée. grad() renvoie une autre fonction, représentant le gradient (c'est-à-dire le vecteur des dérivées partielles) du circuit.

\- \- \-

Thanks to Pennylane, we can access the gradient of the _circuit_ function defined above.

We can differentiate using the built-in grad() function. grad() returns another function, representing the gradient (i.e. vector of partial derivatives) of the circuit.


In [8]:
dcircuit = qml.grad(circuit, argnum=0)
# le circuit prenant de fait un argument "params", on met ici argnum à 0

In [9]:
print(dcircuit([0.54, 0.12]))

[array(-0.51043865), array(-0.1026782)]


### 3) Optimisation - Optimization

PennyLane fournit une collection d'optimiseurs basés sur la descente de gradient. Ces optimiseurs acceptent une fonction de coût et des paramètres initiaux, et utilisent la différenciation automatique de PennyLane pour effectuer une descente de gradient.

Il s'agit ici d'optimiser les deux paramètres $\phi_1$ et $\phi_2$ tel que le qubit, à l'état initial $|0\rangle$ est tourné pour être dans l'état $|1\rangle$.

Pour cela, nous devons définir une fonction de coût. En minimisant la fonction de coût, l'optimiseurs de Pennylane déterminera les valeurs des paramètres du circuit qui permettent d'effectuer la rotation.

\- \- \-

PennyLane provides a collection of gradient descent based optimizers. These optimizers accept a cost function and initial parameters, and use PennyLane's automatic differentiation to perform gradient descent.

This is to optimize the two parameters $\phi_1$ and $\phi_2$ such that the qubit, in the initial state $|0\rangle$ is rotated to be in the state $|1\rangle$ .

For this, we need to define a cost function. By minimizing the cost function, Pennylane's optimizers will determine the values ​​of the circuit parameters that allow the rotation to be performed.

In [10]:
def cost(x):
    return circuit(x)

Pour commencer l'optimisation, choisissons des petites valeurs pour $\phi_1$ et $\phi_2$ :

\- \- \-

To start the optimization, let's choose small values ​​for $\phi_1$ and $\phi_2$:

In [11]:
init_params = np.array([0.011, 0.012], requires_grad = True)
print(cost(init_params))

0.9998675058299389


Notons que pour ces valeurs initiales, la fonction de coût est proche de $1$.

Ainsi, utilisons l'optimiseur afin de mettre à jour les paramètres du circuit sur 100 pas :

\- \- \-

Note that for these initial values, the cost function is close to $1$.

So, let's use the optimizer to update the circuit parameters to 100 steps:

In [12]:
# Initialisation de l'optimiseur
opt = qml.GradientDescentOptimizer(stepsize = 0.4)

# Définition du nombre de pas
steps = 100

# Définition la valeur initiale
params = init_params

for i in range(steps):
    params = opt.step(cost, params)

print("Le résultat de l'optimisation est: {} ".format(params))

Le résultat de l'optimisation est: [7.15266381e-18 3.14159265e+00] 


The optimization result is: [7.15266381e-18 3.14159265e+00]